In [1]:
# !pip install dash
# !pip install jupyter-dash


In [2]:
# !pip install dash-bootstrap-components

In [3]:

import pandas as pd

In [7]:
# import year_earn, buyer form csv file
year_earn = pd.read_csv('data/year_earn.csv')
buyer = pd.read_csv('data/buyer.csv')

In [9]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go

def Header2(name, app):
    title = html.H2(name, style={"margin-top": 5})
    return dbc.Row([dbc.Col(title, md=9)])

def Header1(name, app):
    title = html.H1(name, style={"margin-top": 5})
    return dbc.Row([dbc.Col(title, md=9)])

#데이터 프레임 생성
df = pd.DataFrame({
    'year':[2016, 2017, 2018, 2019, 2020, 2021, 2022],
    'market': [227000000000, 274000000000, 320000000000, 350000000000, 400000000000, 450000000000, 500000000000]
})

년_간편식_HMR__구입_변화_예상_1 = pd.DataFrame({'names': ['현 수준 유지', '약간 증가', '약간 감소', '매우 감소'],
                   'values': [74.9, 22.5, 2.4, 0.2]})

#id=update_graph2, fig3
fig3 = px.pie(년_간편식_HMR__구입_변화_예상_1, values=[74.9, 22.5, 2.4, 0.2], names=['현 수준 유지', '약간 증가', '약간 감소', ' 매우 감소'])
fig3.update_traces(textposition='inside', textinfo='percent+label')
fig3.update_layout(
    annotations=[dict(x=0.5, y=1.2, showarrow=False, text='향후 1년 HMR 구입 변화 예상', font=dict(size=20))],
    margin=dict(t=100)
)

# Card components
cards = [
    dbc.Card(
        [
            html.H2(f"{5/45*100}%", className="card-title"),
            html.P("HMR Market Increasing", className="card-text"),
        ],
        body=True,
        color="dark",
        inverse=True,
    ),
    dbc.Card(
        [
            html.H2(f"계산식 미정", className="card-title"),
            html.P("Decline in Company Sales", className="card-text"),
        ],
        body=True,
        color="primary",
        inverse=True,
    ),
]

# Graph components
graphs1 = [[dcc.Graph(id="update_graph"), #1행 1열
            dcc.RangeSlider(df['year'].min(),
                            df['year'].max(),
                            value=[df['year'].min(), df['year'].max()], #수정사항, 슬라이더 초기값(처음값과 끝값)
                            marks={str(year): str(year) for year in df['year'].unique()}, #수정사항 RangeSlider, Slider 모두에 적용
                            id='slider1')]
            , [dcc.Graph(id="update_graph2", figure=fig3)]] #1행 2열

graphs2 = [[dcc.Graph(id='booking-with-graph'), #2행 1열
            dcc.RangeSlider(id='year-slider',
                            min=year_earn['year'].min(),
                            max=year_earn['year'].max(),
                            value=[year_earn['year'].min(), year_earn['year'].max()],
                            marks={str(year): str(year) for year in year_earn['year'].unique()})],
           [dcc.Graph(id='yearly-transaction-graph'), #2행 2열
            dcc.RangeSlider(id='year-slider2',
                            min= buyer['year'].min(),
                            max= buyer['year'].max(),
                            step=None,
                            value=[buyer['year'].min(), buyer['year'].max()],
                            marks={str(year): str(year) for year in buyer['year'].unique()})
            ]]
        
           # dcc.Graph(id='booking-with-graph')]


# rslider1 = [dcc.RangeSlider('slider1'), dcc.RangeSlider('slider2')]


app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        Header1("불량예측 모델을 통한 수익성 개선", app),
        html.Hr(),
        Header2("Basement", app),
        html.Br(),
        dbc.Row([dbc.Col(card) for card in cards]),
        html.Br(),
        dbc.Row([dbc.Col(graph) for graph in graphs1]),
        dbc.Row([dbc.Col(graph) for graph in graphs2]),
    ],
    fluid=False,
)

#그래프 1행 1열 콜백
@app.callback(
    Output(component_id='update_graph', component_property='figure'), 
    Input(component_id='slider1', component_property='value')
)
def update_graph(year):
    filtered_df = df[(df['year'] >= year[0]) & (df['year'] <= year[1])] #수정사항, 슬라이더 값지정하는 임시 df생성
    fig1 = px.bar(filtered_df, x=filtered_df['year'], y=filtered_df['market'])
    fig1.update_layout(title_text="HMR Market Changes",
                      title_font_size=30)
    return(fig1)


#그래프 2행 1열 콜백
@app.callback(
    Output(component_id='booking-with-graph', component_property='figure'), 
    Input(component_id='year-slider', component_property='value')
)
def update_graph2(year):
    filtered_df2 = year_earn[(year_earn['year'] >= year[0]) & (year_earn['year'] <= year[1])]
    fig2 = px.bar(filtered_df2, x='year', y='수주금액')
    fig2.update_layout(title_text="연도별 수주금액추이",title_font_size=30)
    return(fig2)           

#그래프 2행 2열 콜백
@app.callback(
    Output(component_id='yearly-transaction-graph', component_property='figure'),
    Input(component_id='year-slider2', component_property='value')
)
def update_figure(selected_years):
    filtered_df3 = buyer[(buyer['year'] >= selected_years[0]) & (buyer['year'] <= selected_years[1])]
    fig4 = go.Figure(data=[go.Bar(x=filtered_df3['year'], y=filtered_df3['거래처코드'])],
                     layout=go.Layout(title=go.layout.Title(text="연도별 거래건수 변동추이", x=0.5),
                                      xaxis=dict(title='연도'),
                                      yaxis=dict(title='거래건수'),
                                      barmode='group')
                    )
    return fig4
     
if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')
    

Dash is running on http://127.0.0.1:8050/



from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash

import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc

년_간편식_HMR__구입_변화_예상_1 = pd.DataFrame({'names': ['현 수준 유지', '약간 증가', '약간 감소', '매우 감소'],
                   'values': [74.9, 22.5, 2.4, 0.2]})
import plotly.express as px

fig = px.pie(년_간편식_HMR__구입_변화_예상_1, values=[74.9, 22.5, 2.4, 0.2], names=['현 수준 유지', '약간 증가', '약간 감소', ' 매우 감소'])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    annotations=[dict(x=0.5, y=1.2, showarrow=False, text='향후 1년 HMR 구입 변화 예상', font=dict(size=20))],
    margin=dict(t=100)
)
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph', figure=fig)
])

app.run_server(mode='inline')



,
        dcc.RangeSlider(id='year-slider',
                        min=year_earn['year'].min(),
                        max=year_earn['year'].max(),
                        value=[year_earn['year'].min(), year_earn['year'].max()],
                        marks={str(year): str(year) for year in year_earn['year'].unique()}) 